# 

# Web Scraping Notebook Fuente El tiempo

## Importar librerías

Se importan las librerías de interés para el desarrollo del Web Scraping de la fuente El Tiempo

In [1]:
import selenium as sel
from selenium.webdriver.common.by import By
import pandas as pd
import time
import datetime
import base as bs

import json
import re

import os as os
import csv

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Ingresar la empresa a extraer

Empresa con la cual se va a extraer los artículos. Se ingresa la empresa a la cual se quiere ejecutar el Web Scraping

Adicionalmente, se ingresa también la cantidad de páginas de la fuente El Tiempo a la cual se quiere extraer.

Es decir, por ejemplo si se ingresan N páginas, el Web Scraping extraerá todos los artículos que se encuentran en las N páginas

In [2]:
#Ingresa la empresa a la cual se quiere realizar el Web Scraping. Adicionalmente se utiliza en formato de mínusculas
empresa = input("Digite la empresa a extraer: ").lower().strip()

# Si ingresa una empresa que posee dos palabras o más. Reemplaza el espacio al equivalente URL que necesita la fuente El Tiempo
if " " in empresa:
    empresa = empresa.replace(" ", "%20")

# Ingresa la cantidad de páginas a realizar el Web Scraping del El Tiempo
paginas = input("la cantidad de paginas ")

## Selenium y Web Scraping

Activa el navegador de Microsoft Edge como el predeterminado.

Adicionalmente, realiza la recolección de los datos de los artículos desplegados en la búsqueda de El Tiempo

In [1]:
# Abre el Navegador de Edge
try:
    driver = sel.webdriver.Edge()
except:
    cwd = os.getcwd()
    path = os.path.join(cwd, 'msedgedriver.exe')
    path.replace("\\\\", "\\")
    driver = sel.webdriver.Edge(executable_path=path.replace("\\\\", "\\"))
titulares=[]

# Hacer el WS del primer url
bs.obtener_articulos_eltiempo(driver=driver, url=f'https://www.eltiempo.com/buscar?q={empresa}',titulares = titulares,empresa=empresa)

# Realiza el WS de las demás páginas. El número de páginas ingresado anteriormente
for i in range(2,int(paginas) + 1):
        bs.obtener_articulos_eltiempo(driver=driver, url=f'https://www.eltiempo.com/buscar/{i}?q={empresa}',titulares = titulares,empresa=empresa)


NameError: name 'sel' is not defined

### Extrae info de cada URL

In [ ]:
# Inicializa los cookies del navegador 
driver.delete_all_cookies()

# Busca cada articulo y las almacena en la lista de titulares
for tit in titulares:
    
    driver.get(tit['URL'])
    driver.implicitly_wait(10) # Metodología de wait: espera a que el Navegador tenga cargado toda la página exitosamente

    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException)
    contenido = ''
    try :
        html = WebDriverWait(driver,10,ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.XPATH,'.//div[contains(@class,"modulos public-side")]')))
        parrafos = html.find_elements(By.XPATH,'.//p') 

    except:
        try:
            html = driver.find_element(By.XPATH,'.//div[contains(@class,"modulos public-side")]')
            parrafos = html.find_elements(By.XPATH,'.//p')
        except:
            contenido = 'SIN PARRAFOS' # Si no hay contenido en dicho artículo
        else:
            for i in parrafos:
                contenido += i.text

    else:
        try:
            for i in parrafos:
                contenido += i.text
        except:
            print('Error en el último for == el más dentro de "for tit in titulares:" externo')
    
    # agregar contenido al dict de titulares
    tit['Contenido'] = contenido



    # Para agregar el Autor del artículo
    autor_eltiempo = ''
    try:
        autor_eltiempo = driver.find_element(By.XPATH,"//div[(@class='author_data')]/div/a[@class='who']/span[@class='who']").text
    except:
        autor_eltiempo = 'SIN AUTOR'

    if(autor_eltiempo ==''):
        try:
            autor_eltiempo = driver.find_element(By.XPATH,"//div[(@class='author_data')]/div/a[@class='who']/span[@class='who-modulo who']").text
        except:
            autor_eltiempo = 'SIN AUTOR'

    tit['Autor'] = autor_eltiempo



    #Sacar la imagen del artículo

    imagen=''

    try:
        imagen_temp = driver.find_element(By.XPATH, "//div[@class='recurso_apertura']")
    except:
        imagen = 'SIN IMAGEN'
    else:
        imagen = imagen_temp.find_element(By.XPATH, './/img').get_attribute('src')

    tit['Imagen']=imagen

    # URL de Noticias relacionadas
    relNewsUrls = []

    driver.delete_all_cookies() # clear all cookies in scope of session
    
    #agregar lista de URLs de noticias relacionadas
    # tit['RelNewsUrls'] = bs.obtener_articulos_relacionados_eltiempo(driver)


### Para agregar la info en un archivo CSV

In [ ]:
# Cerrar navegador
driver.quit()

In [ ]:
df=pd.DataFrame(titulares)
df['Empresa'] = df['Empresa'].str.replace('%20',' ')